In [1]:
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles(r'.\data\data\names\*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles(r'.\data\data\names\*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['.\\data\\data\\names\\Arabic.txt', '.\\data\\data\\names\\Chinese.txt', '.\\data\\data\\names\\Czech.txt', '.\\data\\data\\names\\Dutch.txt', '.\\data\\data\\names\\English.txt', '.\\data\\data\\names\\French.txt', '.\\data\\data\\names\\German.txt', '.\\data\\data\\names\\Greek.txt', '.\\data\\data\\names\\Irish.txt', '.\\data\\data\\names\\Italian.txt', '.\\data\\data\\names\\Japanese.txt', '.\\data\\data\\names\\Korean.txt', '.\\data\\data\\names\\Polish.txt', '.\\data\\data\\names\\Portuguese.txt', '.\\data\\data\\names\\Russian.txt', '.\\data\\data\\names\\Scottish.txt', '.\\data\\data\\names\\Spanish.txt', '.\\data\\data\\names\\Vietnamese.txt']
Slusarski


In [2]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [3]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [4]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [5]:
input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [6]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.7587, -2.8917, -2.8822, -2.8398, -2.9847, -2.9200, -2.8787, -3.0092,
         -2.8044, -2.8328, -2.8573, -2.9668, -2.8246, -2.9085, -2.9205, -2.9890,
         -2.9043, -2.8919]], grad_fn=<LogSoftmaxBackward0>)


In [7]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Arabic', 0)


In [8]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = German / line = Greenberg
category = French / line = Lesauvage
category = Japanese / line = Kawagishi
category = Korean / line = Yim
category = Czech / line = Simecek
category = Vietnamese / line = Than
category = Arabic / line = Koury
category = Russian / line = Grozdov
category = Polish / line = Wojda
category = Arabic / line = Shamoon


In [9]:
criterion = nn.NLLLoss()

In [10]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [11]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print ``iter`` number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 9s) 2.8065 Lim  / Japanese ✗ (Korean)
10000 10% (0m 18s) 2.7973 Valencia / Russian ✗ (Spanish)
15000 15% (0m 26s) 1.8299 Nigro / Portuguese ✗ (Italian)
20000 20% (0m 35s) 2.8828 Crofts / Portuguese ✗ (English)
25000 25% (0m 43s) 1.2313 Tosto / Italian ✓
30000 30% (0m 53s) 1.6690 Goldman / English ✓
35000 35% (1m 2s) 2.2895 Ubina / Japanese ✗ (Spanish)
40000 40% (1m 10s) 5.2208 Paulis / Greek ✗ (German)
45000 45% (1m 18s) 1.4037 Ritchie / French ✗ (Scottish)
50000 50% (1m 26s) 1.5040 Cabral / Portuguese ✓
55000 55% (1m 35s) 2.4682 Smart / Dutch ✗ (English)
60000 60% (1m 47s) 0.2291 Joubert / French ✓
65000 65% (1m 59s) 0.0717 Frangopoulos / Greek ✓
70000 70% (2m 9s) 0.6455 Bach / Vietnamese ✓
75000 75% (2m 19s) 0.9888 Bumgarner / German ✓
80000 80% (2m 28s) 0.2399 Mao / Chinese ✓
85000 85% (2m 37s) 0.5861 Slazak / Polish ✓
90000 90% (2m 47s) 1.4440 Meisner / German ✓
95000 95% (2m 56s) 0.2150 Tsai  / Korean ✓
100000 100% (3m 5s) 0.2556 Gui / Chinese ✓


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [ ]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('Dovesky')
predict('Jackson')
predict('Satoshi')

In [ ]:
predict('Park')

In [ ]:
predict('Carpenter')

In [ ]:
predict('Oliver')

In [ ]:
torch.save(rnn.state_dict(), r'C:\Users\brian\OneDrive - University of Tennessee\Desktop\Research\Python program\Our_website\assets\char-rnn-classification.pht')